<a href="https://colab.research.google.com/github/jmcaussade/Homologacion-Car-Makers/blob/main/NHTSA_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rapidfuzz

In [1]:
import pandas as pd
import requests
import time
from rapidfuzz.fuzz import ratio

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import files

In [4]:
ruta_data_set = '/content/drive/MyDrive/Xinergy/Proyecto_homologacion_autos/database.xlsx'
df = pd.read_excel(ruta_data_set, sheet_name='Anexo 6')
df.head()
df

,Unnamed: 0,numcaso,tipo_vehiculo,fecha_servicio,plate,serie,model,Tipo de Auto,car_brand,car_model,car_type,Peso kg
0,NaN,355845,Auto,2023-09-01 00:01:13,SZD861A,3HGCM66563G001615,2003.0,estandar,Honda,ACCORD,Auto,NaN
1,NaN,355847,Camion,2023-09-01 00:06:09,TP9498G,3N6DD25T2EK005444,2014.0,estandar,NISSAN,ESTACAS,camion,NaN
2,NaN,355849,Auto,2023-09-01 00:08:38,NCZ1892,MEX5G2603JT120368,2018.0,estandar,Volkswagen,VENTO,Auto,NaN
3,NaN,355857,Auto,2023-09-01 00:24:34,FCJ597B,1C4BJWEG6FL629655,2015.0,estandar,JEEP,J WRANGLER,Auto,NaN
4,NaN,355864,Auto,2023-09-01 00:46:09,NJA2747,MA6CB6CD6KT029772,2019.0,estandar,CHEVROLET,BEAT,Auto,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
279424,NaN,745808,Equipo Ligero,2024-08-31 00:00:00,TDY040B,3N1CN7AD7KK399688,2019.0,NaN,NISSAN,VERSA,Equipo Ligero,NaN
279425,NaN,745810,Equipo Ligero,2024-08-31 00:00:00,JUB4510,LSJA24U99RN010717,2024.0,NaN,MG,HS,Equipo Ligero,NaN
279426,NaN,745811,Equipo Ligero,2024-08-31 00:00:00,RTR920A,3G1SF21X97S112886,2007.0,NaN,CHEVROLET,CHEVY,Equipo Ligero,NaN
279427,NaN,745814,Equipo Ligero,2024-08-31 00:00:00,PWEE63E,LJD5AA194R0124080,2024.0,NaN,KIA,SELTOS,Equipo Ligero,NaN


In [34]:
df["clean_serie"] = df["serie"].fillna("").astype(str).str.strip()

In [35]:
df

,Unnamed: 0,numcaso,tipo_vehiculo,fecha_servicio,plate,serie,model,Tipo de Auto,car_brand,car_model,car_type,Peso kg,clean_serie,clean_serie_length
0,NaN,355845,Auto,2023-09-01 00:01:13,SZD861A,3HGCM66563G001615,2003.0,estandar,Honda,ACCORD,Auto,NaN,3HGCM66563G001615,17
1,NaN,355847,Camion,2023-09-01 00:06:09,TP9498G,3N6DD25T2EK005444,2014.0,estandar,NISSAN,ESTACAS,camion,NaN,3N6DD25T2EK005444,17
2,NaN,355849,Auto,2023-09-01 00:08:38,NCZ1892,MEX5G2603JT120368,2018.0,estandar,Volkswagen,VENTO,Auto,NaN,MEX5G2603JT120368,17
3,NaN,355857,Auto,2023-09-01 00:24:34,FCJ597B,1C4BJWEG6FL629655,2015.0,estandar,JEEP,J WRANGLER,Auto,NaN,1C4BJWEG6FL629655,17
4,NaN,355864,Auto,2023-09-01 00:46:09,NJA2747,MA6CB6CD6KT029772,2019.0,estandar,CHEVROLET,BEAT,Auto,NaN,MA6CB6CD6KT029772,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279424,NaN,745808,Equipo Ligero,2024-08-31 00:00:00,TDY040B,3N1CN7AD7KK399688,2019.0,NaN,NISSAN,VERSA,Equipo Ligero,NaN,3N1CN7AD7KK399688,17
279425,NaN,745810,Equipo Ligero,2024-08-31 00:00:00,JUB4510,LSJA24U99RN010717,2024.0,NaN,MG,HS,Equipo Ligero,NaN,LSJA24U99RN010717,17
279426,NaN,745811,Equipo Ligero,2024-08-31 00:00:00,RTR920A,3G1SF21X97S112886,2007.0,NaN,CHEVROLET,CHEVY,Equipo Ligero,NaN,3G1SF21X97S112886,17
279427,NaN,745814,Equipo Ligero,2024-08-31 00:00:00,PWEE63E,LJD5AA194R0124080,2024.0,NaN,KIA,SELTOS,Equipo Ligero,NaN,LJD5AA194R0124080,17


In [36]:
# Add a new column that counts the number of characters in 'clean_serie'
df["clean_serie_length"] = df["clean_serie"].apply(len)


In [82]:
df2 = df[df["clean_serie_length"] == 17].sample(n=10000, random_state=42)
# Randomly selects 1,000 rows


In [83]:
df2

,Unnamed: 0,numcaso,tipo_vehiculo,fecha_servicio,plate,serie,model,Tipo de Auto,car_brand,car_model,car_type,Peso kg,clean_serie,clean_serie_length
149377,NaN,630462,Auto,2024-05-13 17:29:53,PDH7358,3N1CN8AE4MK863814,2021.0,estandar,NISSAN,VERSA,Auto,NaN,3N1CN8AE4MK863814,17
90892,NaN,522331,Auto,2024-01-31 22:25:09,FBX696B,JM1BK123081818589,2008.0,estandar,Mazda,3,Auto,NaN,JM1BK123081818589,17
97633,NaN,535052,Auto,2024-02-13 16:38:55,L10AXZ,LUCGM654XJ3116979,2018.0,estandar,Honda,CITY,Auto,NaN,LUCGM654XJ3116979,17
154181,NaN,639175,Auto,2024-05-21 15:37:16,P23ASD,VSSAC4NH7J1006213,2018.0,estandar,SEAT,TOLEDO,Auto,NaN,VSSAC4NH7J1006213,17
27305,NaN,405007,Auto,2023-10-13 10:26:08,NEL4J,1M04X2XDEHM120122,2017.0,estandar,MOTOCICLETAS,MOTOCICLETAS,Auto,NaN,1M04X2XDEHM120122,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226902,NaN,430449,Equipo Ligero,2023-11-04 00:00:00,SPN504A,3vw2w2aj5jm222698,2018.0,NaN,VOLKSWAGEN,JETTA MK VI,Equipo Ligero,NaN,3vw2w2aj5jm222698,17
197374,NaN,714669,Auto,2024-08-01 09:59:10,s/n,1G1Z95ST5GF209333,2016.0,estandar,CHEVROLET,MALIBU,Auto,NaN,1G1Z95ST5GF209333,17
166412,NaN,660893,Auto,2024-06-10 14:15:49,JGR4668,3VWRV09M78M640405,2008.0,estandar,Volkswagen,Jetta,Auto,NaN,3VWRV09M78M640405,17
155523,NaN,641555,Auto,2024-05-23 17:34:57,VDM407B,3N1BC1AS8CK245442,2012.0,estandar,NISSAN,TIIDA,Auto,NaN,3N1BC1AS8CK245442,17


In [84]:
df2["Maker_api"] = None  # Add a new column for results
df2["Model_api"] = None  # Add a new column for results
df2["Type_api"] = None
df2["Weight_api"] = None  # Add a new column for results

In [85]:
df2

,Unnamed: 0,numcaso,tipo_vehiculo,fecha_servicio,plate,serie,model,Tipo de Auto,car_brand,car_model,car_type,Peso kg,clean_serie,clean_serie_length,Maker_api,Model_api,Type_api,Weight_api
149377,NaN,630462,Auto,2024-05-13 17:29:53,PDH7358,3N1CN8AE4MK863814,2021.0,estandar,NISSAN,VERSA,Auto,NaN,3N1CN8AE4MK863814,17,None,None,None,None
90892,NaN,522331,Auto,2024-01-31 22:25:09,FBX696B,JM1BK123081818589,2008.0,estandar,Mazda,3,Auto,NaN,JM1BK123081818589,17,None,None,None,None
97633,NaN,535052,Auto,2024-02-13 16:38:55,L10AXZ,LUCGM654XJ3116979,2018.0,estandar,Honda,CITY,Auto,NaN,LUCGM654XJ3116979,17,None,None,None,None
154181,NaN,639175,Auto,2024-05-21 15:37:16,P23ASD,VSSAC4NH7J1006213,2018.0,estandar,SEAT,TOLEDO,Auto,NaN,VSSAC4NH7J1006213,17,None,None,None,None
27305,NaN,405007,Auto,2023-10-13 10:26:08,NEL4J,1M04X2XDEHM120122,2017.0,estandar,MOTOCICLETAS,MOTOCICLETAS,Auto,NaN,1M04X2XDEHM120122,17,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226902,NaN,430449,Equipo Ligero,2023-11-04 00:00:00,SPN504A,3vw2w2aj5jm222698,2018.0,NaN,VOLKSWAGEN,JETTA MK VI,Equipo Ligero,NaN,3vw2w2aj5jm222698,17,None,None,None,None
197374,NaN,714669,Auto,2024-08-01 09:59:10,s/n,1G1Z95ST5GF209333,2016.0,estandar,CHEVROLET,MALIBU,Auto,NaN,1G1Z95ST5GF209333,17,None,None,None,None
166412,NaN,660893,Auto,2024-06-10 14:15:49,JGR4668,3VWRV09M78M640405,2008.0,estandar,Volkswagen,Jetta,Auto,NaN,3VWRV09M78M640405,17,None,None,None,None
155523,NaN,641555,Auto,2024-05-23 17:34:57,VDM407B,3N1BC1AS8CK245442,2012.0,estandar,NISSAN,TIIDA,Auto,NaN,3N1BC1AS8CK245442,17,None,None,None,None


In [86]:
# API Function
def get_vehicle_info(vin):
    """
    Fetches the maker, model, weight, and vehicle type for a given VIN using the NHTSA API in a single request.

    Parameters:
    vin (str): The VIN of the vehicle.

    Returns:
    dict: A dictionary containing 'Maker', 'Model', 'Weight', and 'Vehicle Type' keys with their corresponding values, or None if not found.
    """
    url = f"https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVin/{vin}?format=json"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            results = response.json().get("Results", [])
            maker = None
            model = None
            vehicle_type = None
            weight = None

            # Loop through results to extract relevant data
            for item in results:
                if item["Variable"] == "Make" and item["Value"]:
                    maker = item["Value"]
                elif item["Variable"] == "Model" and item["Value"]:
                    model = item["Value"]
                elif item["Variable"] == "Vehicle Type" and item["Value"]:
                    vehicle_type = item["Value"]
                elif item["Variable"] == "Gross Vehicle Weight Rating From" and item["Value"]:
                    weight = item["Value"]

            return {
                "Maker": maker,
                "Model": model,
                "Vehicle Type": vehicle_type,
                "Weight": weight
            }

        return None  # Return None if the API call is unsuccessful
    except Exception as e:
        print(f"Error for VIN {vin}: {e}")
        return None


In [87]:
# Save progress every BATCH_SIZE rows
BATCH_SIZE = 1000
OUTPUT_FILE = "/content/drive/MyDrive/Xinergy/Proyecto_homologacion_autos/vehicle_info_progress.xlsx"

# Reload progress if the file exists
try:
    # Load the progress file
    progress_df = pd.read_excel(OUTPUT_FILE)
    print(f"Loaded progress from {OUTPUT_FILE}")

    # Merge progress with the original DataFrame
    df2 = df2.merge(progress_df, how="left", on="clean_serie", suffixes=("", "_progress"))

    # Fill columns from progress
    for col in ["Maker_api", "Model_api", "Type_api", "Weight_api"]:
        df2[col] = df2[f"{col}_progress"].combine_first(df2[col])

    # Drop extra columns created during merging
    df2 = df2.drop(columns=[col for col in df2.columns if col.endswith("_progress")])

except FileNotFoundError:
    print(f"No progress file found. Starting fresh.")


No progress file found. Starting fresh.


In [88]:
# Iterate through DataFrame
for index, row in df2.iterrows():
    # Skip already processed rows
    if pd.notnull(row.get("Maker_api")):
        continue

    vin = row["clean_serie"]

    if len(vin) != 17:  # Skip rows where VIN is not 17 characters
        continue

    # Fetch all vehicle information in a single API call
    vehicle_info = get_vehicle_info(vin)

    if vehicle_info:
        # Update respective columns with fetched information in lowercase
        df2.at[index, "Maker_api"] = vehicle_info["Maker"].lower() if vehicle_info["Maker"] else None
        df2.at[index, "Model_api"] = vehicle_info["Model"].lower() if vehicle_info["Model"] else None
        df2.at[index, "Type_api"] = vehicle_info["Vehicle Type"].lower() if vehicle_info["Vehicle Type"] else None
        df2.at[index, "Weight_api"] = vehicle_info["Weight"].lower() if vehicle_info["Weight"] else None


        print(f"Processed VIN {index}: {vin} -> Maker: {vehicle_info['Maker']}, Model: {vehicle_info['Model']}, Vehicle Type: {vehicle_info['Vehicle Type']} Weight: {vehicle_info['Weight']}")
    else:
        print(f"Failed to retrieve data for VIN {index}: {vin}")

    # Save progress every BATCH_SIZE rows
    if (index + 1) % BATCH_SIZE == 0:
        df2.to_excel(OUTPUT_FILE, index=False)
        print(f"Progress saved at row {index + 1} to {OUTPUT_FILE}")

    # Add a delay to respect API rate limits
    time.sleep(0.1)

# Final save after loop completion
df2.to_excel(OUTPUT_FILE, index=False)
print(f"Processing complete. Final progress saved to {OUTPUT_FILE}")


Se han truncado las últimas 5000 líneas del flujo de salida.
Processed VIN 261831: JH4CU26689C400083 -> Maker: ACURA, Model: TSX, Vehicle Type: PASSENGER CAR Weight: Class 1C: 4,001 - 5,000 lb (1,814 - 2,268 kg)
Processed VIN 176933: 3N1AB8AE3NY261470 -> Maker: None, Model: None, Vehicle Type: PASSENGER CAR Weight: None
Processed VIN 86567: 1FTRE1421XHC08457 -> Maker: FORD, Model: E-150, Vehicle Type: TRUCK Weight: Class 2E: 6,001 - 7,000 lb (2,722 - 3,175 kg)
Processed VIN 235135: 9BHCP5AC8PP424992 -> Maker: None, Model: None, Vehicle Type: None Weight: None
Processed VIN 197734: 1HGFA158X8L900444 -> Maker: HONDA, Model: Civic, Vehicle Type: PASSENGER CAR Weight: Class 1B: 3,001 - 4,000 lb (1,360 - 1,814 kg)
Processed VIN 221463: MB2PB2A23NM334113 -> Maker: None, Model: None, Vehicle Type: None Weight: None
Processed VIN 157381: KL1JM52Z98K305070 -> Maker: CHEVROLET, Model: None, Vehicle Type: PASSENGER CAR Weight: None
Processed VIN 183460: 3SCK5FLD7N1000676 -> Maker: None, Model: No

In [89]:
df2

,Unnamed: 0,numcaso,tipo_vehiculo,fecha_servicio,plate,serie,model,Tipo de Auto,car_brand,car_model,car_type,Peso kg,clean_serie,clean_serie_length,Maker_api,Model_api,Type_api,Weight_api
149377,NaN,630462,Auto,2024-05-13 17:29:53,PDH7358,3N1CN8AE4MK863814,2021.0,estandar,NISSAN,VERSA,Auto,NaN,3N1CN8AE4MK863814,17,None,None,passenger car,None
90892,NaN,522331,Auto,2024-01-31 22:25:09,FBX696B,JM1BK123081818589,2008.0,estandar,Mazda,3,Auto,NaN,JM1BK123081818589,17,mazda,mazda3,passenger car,"class 1: 6,000 lb or less (2,722 kg or less)"
97633,NaN,535052,Auto,2024-02-13 16:38:55,L10AXZ,LUCGM654XJ3116979,2018.0,estandar,Honda,CITY,Auto,NaN,LUCGM654XJ3116979,17,None,None,None,None
154181,NaN,639175,Auto,2024-05-21 15:37:16,P23ASD,VSSAC4NH7J1006213,2018.0,estandar,SEAT,TOLEDO,Auto,NaN,VSSAC4NH7J1006213,17,None,None,None,None
27305,NaN,405007,Auto,2023-10-13 10:26:08,NEL4J,1M04X2XDEHM120122,2017.0,estandar,MOTOCICLETAS,MOTOCICLETAS,Auto,NaN,1M04X2XDEHM120122,17,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226902,NaN,430449,Equipo Ligero,2023-11-04 00:00:00,SPN504A,3vw2w2aj5jm222698,2018.0,NaN,VOLKSWAGEN,JETTA MK VI,Equipo Ligero,NaN,3vw2w2aj5jm222698,17,volkswagen,jetta,passenger car,"class 1: 6,000 lb or less (2,722 kg or less)"
197374,NaN,714669,Auto,2024-08-01 09:59:10,s/n,1G1Z95ST5GF209333,2016.0,estandar,CHEVROLET,MALIBU,Auto,NaN,1G1Z95ST5GF209333,17,chevrolet,malibu,passenger car,"class 1: 6,000 lb or less (2,722 kg or less)"
166412,NaN,660893,Auto,2024-06-10 14:15:49,JGR4668,3VWRV09M78M640405,2008.0,estandar,Volkswagen,Jetta,Auto,NaN,3VWRV09M78M640405,17,volkswagen,None,passenger car,None
155523,NaN,641555,Auto,2024-05-23 17:34:57,VDM407B,3N1BC1AS8CK245442,2012.0,estandar,NISSAN,TIIDA,Auto,NaN,3N1BC1AS8CK245442,17,nissan,versa,passenger car,"class 1: 6,000 lb or less (2,722 kg or less)"


# Etapa de estandarización de marca y modelo

In [90]:
df2["clean_car_brand"] = (
    df2["car_brand"]
    .fillna("")  # Replace NaN with an empty string
    .astype(str)  # Ensure all values are strings
    .str.lower()  # Convert all strings to lowercase
    .str.replace(r'[^a-z0-9 ]', '', regex=True)  # Remove only special characters
    .str.strip()  # Remove leading and trailing spaces
)

In [91]:
df2["Similarity_score"] = None
df2["Maker_final"] = None

In [92]:
df2

,Unnamed: 0,numcaso,tipo_vehiculo,fecha_servicio,plate,serie,model,Tipo de Auto,car_brand,car_model,...,Peso kg,clean_serie,clean_serie_length,Maker_api,Model_api,Type_api,Weight_api,clean_car_brand,Similarity_score,Maker_final
149377,NaN,630462,Auto,2024-05-13 17:29:53,PDH7358,3N1CN8AE4MK863814,2021.0,estandar,NISSAN,VERSA,...,NaN,3N1CN8AE4MK863814,17,None,None,passenger car,None,nissan,None,None
90892,NaN,522331,Auto,2024-01-31 22:25:09,FBX696B,JM1BK123081818589,2008.0,estandar,Mazda,3,...,NaN,JM1BK123081818589,17,mazda,mazda3,passenger car,"class 1: 6,000 lb or less (2,722 kg or less)",mazda,None,None
97633,NaN,535052,Auto,2024-02-13 16:38:55,L10AXZ,LUCGM654XJ3116979,2018.0,estandar,Honda,CITY,...,NaN,LUCGM654XJ3116979,17,None,None,None,None,honda,None,None
154181,NaN,639175,Auto,2024-05-21 15:37:16,P23ASD,VSSAC4NH7J1006213,2018.0,estandar,SEAT,TOLEDO,...,NaN,VSSAC4NH7J1006213,17,None,None,None,None,seat,None,None
27305,NaN,405007,Auto,2023-10-13 10:26:08,NEL4J,1M04X2XDEHM120122,2017.0,estandar,MOTOCICLETAS,MOTOCICLETAS,...,NaN,1M04X2XDEHM120122,17,None,None,None,None,motocicletas,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226902,NaN,430449,Equipo Ligero,2023-11-04 00:00:00,SPN504A,3vw2w2aj5jm222698,2018.0,NaN,VOLKSWAGEN,JETTA MK VI,...,NaN,3vw2w2aj5jm222698,17,volkswagen,jetta,passenger car,"class 1: 6,000 lb or less (2,722 kg or less)",volkswagen,None,None
197374,NaN,714669,Auto,2024-08-01 09:59:10,s/n,1G1Z95ST5GF209333,2016.0,estandar,CHEVROLET,MALIBU,...,NaN,1G1Z95ST5GF209333,17,chevrolet,malibu,passenger car,"class 1: 6,000 lb or less (2,722 kg or less)",chevrolet,None,None
166412,NaN,660893,Auto,2024-06-10 14:15:49,JGR4668,3VWRV09M78M640405,2008.0,estandar,Volkswagen,Jetta,...,NaN,3VWRV09M78M640405,17,volkswagen,None,passenger car,None,volkswagen,None,None
155523,NaN,641555,Auto,2024-05-23 17:34:57,VDM407B,3N1BC1AS8CK245442,2012.0,estandar,NISSAN,TIIDA,...,NaN,3N1BC1AS8CK245442,17,nissan,versa,passenger car,"class 1: 6,000 lb or less (2,722 kg or less)",nissan,None,None


In [93]:
# Variable for minimum fuzz score
min_value_fuzz_score = 60  # Set this value as needed

# List of error strings to check
list_errors = ["legalizado", "fronterizo", "especiales"]

# Iterate through the DataFrame and apply the conditions
for index, row in df2.iterrows():
    maker_api = row["Maker_api"]
    clean_car_brand = row["clean_car_brand"]

    # Case 1: No value in "clean_car_brand" but value in "Maker_api"
    if pd.isna(clean_car_brand) and pd.notna(maker_api):
        df2.at[index, "Maker_final"] = maker_api

    # Case 2: Value in "clean_car_brand" but no value in "Maker_api"
    elif pd.notna(clean_car_brand) and pd.isna(maker_api):
        df2.at[index, "Maker_final"] = clean_car_brand

    # Case 3: Both "clean_car_brand" and "Maker_api" have values
    elif pd.notna(clean_car_brand) and pd.notna(maker_api):
        # Check if "clean_car_brand" contains any of the error strings
        if any(error in clean_car_brand.lower() for error in list_errors):
            df2.at[index, "Maker_final"] = maker_api
        else:
            # Calculate the fuzz ratio
            fuzz_score = ratio(clean_car_brand, maker_api)
            df2.at[index, "Similarity_score"] = fuzz_score

            # Set "Maker_final" based on the fuzz score
            if fuzz_score >= min_value_fuzz_score:
                df2.at[index, "Maker_final"] = maker_api
            else:
                df2.at[index, "Maker_final"] = "N/A"


In [94]:
df2

,Unnamed: 0,numcaso,tipo_vehiculo,fecha_servicio,plate,serie,model,Tipo de Auto,car_brand,car_model,...,Peso kg,clean_serie,clean_serie_length,Maker_api,Model_api,Type_api,Weight_api,clean_car_brand,Similarity_score,Maker_final
149377,NaN,630462,Auto,2024-05-13 17:29:53,PDH7358,3N1CN8AE4MK863814,2021.0,estandar,NISSAN,VERSA,...,NaN,3N1CN8AE4MK863814,17,None,None,passenger car,None,nissan,None,nissan
90892,NaN,522331,Auto,2024-01-31 22:25:09,FBX696B,JM1BK123081818589,2008.0,estandar,Mazda,3,...,NaN,JM1BK123081818589,17,mazda,mazda3,passenger car,"class 1: 6,000 lb or less (2,722 kg or less)",mazda,100.0,mazda
97633,NaN,535052,Auto,2024-02-13 16:38:55,L10AXZ,LUCGM654XJ3116979,2018.0,estandar,Honda,CITY,...,NaN,LUCGM654XJ3116979,17,None,None,None,None,honda,None,honda
154181,NaN,639175,Auto,2024-05-21 15:37:16,P23ASD,VSSAC4NH7J1006213,2018.0,estandar,SEAT,TOLEDO,...,NaN,VSSAC4NH7J1006213,17,None,None,None,None,seat,None,seat
27305,NaN,405007,Auto,2023-10-13 10:26:08,NEL4J,1M04X2XDEHM120122,2017.0,estandar,MOTOCICLETAS,MOTOCICLETAS,...,NaN,1M04X2XDEHM120122,17,None,None,None,None,motocicletas,None,motocicletas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226902,NaN,430449,Equipo Ligero,2023-11-04 00:00:00,SPN504A,3vw2w2aj5jm222698,2018.0,NaN,VOLKSWAGEN,JETTA MK VI,...,NaN,3vw2w2aj5jm222698,17,volkswagen,jetta,passenger car,"class 1: 6,000 lb or less (2,722 kg or less)",volkswagen,100.0,volkswagen
197374,NaN,714669,Auto,2024-08-01 09:59:10,s/n,1G1Z95ST5GF209333,2016.0,estandar,CHEVROLET,MALIBU,...,NaN,1G1Z95ST5GF209333,17,chevrolet,malibu,passenger car,"class 1: 6,000 lb or less (2,722 kg or less)",chevrolet,100.0,chevrolet
166412,NaN,660893,Auto,2024-06-10 14:15:49,JGR4668,3VWRV09M78M640405,2008.0,estandar,Volkswagen,Jetta,...,NaN,3VWRV09M78M640405,17,volkswagen,None,passenger car,None,volkswagen,100.0,volkswagen
155523,NaN,641555,Auto,2024-05-23 17:34:57,VDM407B,3N1BC1AS8CK245442,2012.0,estandar,NISSAN,TIIDA,...,NaN,3N1BC1AS8CK245442,17,nissan,versa,passenger car,"class 1: 6,000 lb or less (2,722 kg or less)",nissan,100.0,nissan


#Export

In [95]:
# Define the file name and path
output_file = "df2_export_of.xlsx"

# Export df2 to an Excel file
df2.to_excel(output_file, index=False)

print(f"DataFrame exported successfully to {output_file}")


DataFrame exported successfully to df2_export_of.xlsx
